In [1]:
import numpy as np
from sympy import symbols as syb
from sympy import MatrixSymbol as msyb
from sympy import sqrt, Array, I, Matrix

In [2]:
from sympy.physics.quantum import Bra,Ket, TensorProduct
from sympy.physics.quantum.gate import HadamardGate as H
from sympy.physics.quantum.gate import ZGate as Z
from sympy import simplify, Eq, linsolve, solve
from sympy.solvers import solve, nsolve

- $|\psi(0)\rangle = \frac{1}{\sqrt{2E}} \begin{bmatrix} \sqrt{M}\dot{\vec{x}}(0) \\ i\vec{\mu}(0) \end{bmatrix} \text{ and  } |\psi(t)\rangle = \frac{1}{\sqrt{2E}} \begin{bmatrix} \sqrt{M}\dot{\vec{x}}(t) \\ i\vec{\mu}(t) \end{bmatrix} $ where $dim(|\psi\rangle) = N + M | M = N(N+1)/2$

- Say $n = \log_2 N$

- I have assumed that the relative phase in the state only differ by $k\pi$, where $k$ is an integer, which is a fair assumption, as velocities and positions are are real terms.

- Another assumption I made is that, the state is separable in Kinetic Energy and potential energy qubits, hence, I can apply projective measurement separately to both part. This again is justifiable as because of how the state encding remains throughout the simulation.

- For only getiing the velocities, I require Computational basis measurement of all KE qubits and $X^{\otimes n}$ basis measurement of KE qubits. Which makes the complexity as $$\mathcal{O}()+\mathcal{O}(2)$$ which is a constant; as compared to the complexity for traditional state tomography which will be $$\mathcal{O}()+\mathcal{O}(4^n).$$ Exponential in n



#### For a quantum state of arbitrary size

##### This cell sets up the common initial system and initial state of that system

In [3]:
#defining intial system globally maybe the instance of a class
N = 2
M = [1]*N
E = 1
#K = msyb('K', N, N)
k11 = 1
k12 = 1
k23 = 1
k34 = 1
K = np.array(([k11,k12],[k12,k23]))
x = list(syb('x0:%d'%N,real = True)) #unknowns postion vector
x_dot = list(syb('v0:%d'%N,real = True)) #unknown velocity vector

In [4]:
#initial guess
x_ini = [1,-1]
x_dot_ini = [0,0]

##### Actual post-processing code to extract the velocities and posititons from output probabilities.

In [5]:
def dummy_state():

  # N(N+1)/2 terms, potential energy part of the quantum state
  mu = []
  for i in range(N):
      for j in range(N):
          if j>i:
              mu.append((I*sqrt(K[i,j])*(x[i]-x[j]))/sqrt(2*E))
          elif j==i:
              mu.append((I*sqrt(K[j,j])*x[j])/sqrt(2*E))
  # N terms kinetic energy part of the quantum state
  y_dot = []
  for i in range(N):
    y_dot.append(x_dot[i]*sqrt(M[i])/sqrt(2*E))
    psi0 = y_dot+mu
  #padding with zeros
  for i in range(int(3*N*(N-1)/2)):
    psi0.append(0)
  return psi0, mu,

def __get_ke_part(psi,N):
  return psi[:N]     #extracted the kitentic part of the state

def x_measure_equations(psi,N):
  psi0_ext = __get_ke_part(psi,N) #get the KE part
  q = int(np.log2(N))    #number of qubits carrying kinetic energy terms
  gate = H(q)
  H_matrix = gate.get_target_matrix()
  # Create the tensor product of the Hadamard gate for n qubits
  H_otimes_q = TensorProduct(*[H_matrix for _ in range(q)])
  phi0_ext = H_otimes_q*Matrix(psi0_ext)
  phi_amps = []
  for i in range(len(phi0_ext)):
    phi_amps.append(phi0_ext[i]*phi0_ext[i])
  return phi_amps

def post_process_ke(amps_list,out_probs_x,N):
  Eq_list = []
  for i in range(N):
    Eq_list.append(simplify(amps_list[i])-out_probs_x[i])
  print(Eq_list)
  solution = solve(Eq_list, x_dot,dict =True)
  return {"Equations":Eq_list, "Solutions": solution}

def eliminate_solution(out_probs_comp,solutions):
  filtered_sols = []
  for cases in solutions:
    # Check if all values in the dictionary match the corresponding compare value
    if np.allclose(out_probs_comp,[float(_) for _ in np.array(list(cases.values()))**2/2]):
      filtered_sols.append(cases)
  return filtered_sols

#### Now to deal with position part

In [6]:
def __get_pe_part(psi,N):
  return psi[N:int(N*(N+3)/2)]     #extracted the kitentic part of the state

def eq_association(mu,out_probs_pe=None):
   lst = simplify(abs(np.array(mu))*abs(np.array(mu)))
   Eq_list = []
   for i in range(len(lst)):
    if lst[i] != 0: #removing zeros in equations because of zeros in K matrix
      Eq_list.append(lst[i]-out_probs_pe[i])
   return Eq_list

In [7]:
def post_process_pe(mu, out_probs):
  out_probs_pe = out_probs[N:int(N*(N+3)/2)]
  Eq_list = eq_association(mu,out_probs_pe)
  print(Eq_list[0:4],x[0:4])
  solution = solve(Eq_list,x ,dict =True)
  return solution

###### An Example for N = 2

In [8]:
dummy, mu = dummy_state()

In [9]:
dummy

[sqrt(2)*v0/2,
 sqrt(2)*v1/2,
 sqrt(2)*I*x0/2,
 sqrt(2)*I*(x0 - x1)/2,
 sqrt(2)*I*x1/2,
 0,
 0,
 0]

In [18]:
# Now let's suppose the probabilities obtained after Hamiltonian simulation and measurement of state in both hadamard basis and computational basis are:
out_probs_comp = np.array(([1/8,1/8,1/2,1/8,1/8,0,0,0]))
out_probs_x = np.array(([0,1/4,1/4,1/2,0,0,0,0]))

In [22]:
#to compute the positions from the potential energy part we only need the output probabilities from computational basis measurement
post_process_pe(mu, out_probs_comp)

[x0**2/2 - 0.5, (x0 - x1)**2/2 - 0.125, x1**2/2 - 0.125] [x0, x1]


[{x0: -1.00000000000000, x1: -0.500000000000000},
 {x0: 1.00000000000000, x1: 0.500000000000000}]

In [24]:
#to compute the velocities from kinetic energy part we need both computational basis measurement outcome and hadamard basis outcome

#first we generate the kinetic energy part of equations corresponding to system state in hadamard basis
amps_list = x_measure_equations(dummy,2)

#then use hadamard basis measurement and the equation to get possible solutions
post_process_out = post_process_ke(amps_list,out_probs_x,2)

[(v0 + v1)**2/4, (v0 - v1)**2/4 - 0.25]


In [29]:
#finally we use kinetic part of the measurement outcomes in computational basis to eliminate incorrect solutions
eliminate_solution(out_probs_comp[0:2],post_process_out["Solutions"])

[{v0: -0.500000000000000, v1: 0.500000000000000},
 {v0: 0.500000000000000, v1: -0.500000000000000}]